In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.spatial.distance as dist
import itertools

import iblofunmatch.inter as ibfm
output_dir = "output" 
plots_dir = "plots/example_computation/"

import os 
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

# Computation of Block Function in dimension 0

Consider the following example, with points taken from a sample.

We consider 7 points and a sample of three points. 

In [ ]:
RandGen = np.random.default_rng(2)
X = ibfm.sampled_circle(0,2,6, RandGen)
S_indices = RandGen.choice(X.shape[0],3, replace=False)
S = X[S_indices]
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0.png")

Next, we compute the block function induced by the inclusion $S\hookrightarrow X$

In [ ]:
ibfm_out = ibfm.get_IBloFunMatch_subset(None, X, S_indices, output_dir, num_it=4, max_rad=-1, points=True, store_0_pm=True)

In [ ]:
ibfm_out["S_barcode_0"]

In [ ]:
ibfm_out["X_barcode_0"]

In [ ]:
ibfm_out["pm_matrix_0"]

In [ ]:
ibfm_out["block_function_0"]

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_matching(ibfm_out, ax, fig, block_function=True, dim=0)
plt.savefig(plots_dir + "block_function_0.png")

## Geometric intuition for previous computation

Now, we perform the previous computation in terms of connected components. First, we plot again the points with numbers besides them.

In [ ]:
S = X[S_indices]
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
for idx in range(X.shape[0]):
    ax.text(X[idx,0]+0.05, X[idx,1]+0.05, f"{idx}", fontsize=10)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0_numbered.png")

Denote by $a_1, a_2$ the endpoints from $\textrm{PH}_0(X)$ such that $a_1 < a_2$. Also, denote $b_1 < \cdots < b_5$ the endpoints from the barcode of $\textrm{PH}_0(Z)$.

Now, we show that $\mathcal{M}_f(a_1, b_5) = \textrm{dim}(\textrm{H}_0(G(X,Z)_{a_1,b_5}) = 1$

In [ ]:
def compute_components(edgelist, num_points):
    components = np.array(range(num_points))
    for edge in edgelist:
        indices = np.nonzero(components == components[np.max(edge)])[0]
        components[indices]=np.ones(len(indices))*components[np.min(edge)]
    return components

In [ ]:
def plot_geometric_matching(a, b, idx_S, X, ibfm_out, ax, _tol=1e-5):
    S = X[idx_S]
    # Obtain indices of bars that are approximately equal to a and b, these go from (a_idx - a_shift) to a_idx. (same for b_idx)
    a_idx = np.max(np.nonzero(ibfm_out["S_barcode_0"][:,1] < a + _tol))
    a_shift = np.sum(ibfm_out["S_barcode_0"][:,1][:a_idx+1] > a - _tol)
    b_idx = np.max(np.nonzero(ibfm_out["X_barcode_0"][:,1] < b + _tol))
    b_shift = np.sum(ibfm_out["X_barcode_0"][:,1][:b_idx+1] > b - _tol)
    print(f"a_idx:{a_idx}, a_shift:{a_shift}, b_idx:{b_idx}, b_shift:{b_shift}")
    pair_ab = [a_idx, b_idx]
    shift_ab = [a_shift, b_shift]
    num_points = X.shape[0]
    for idx in range(3):
        ax[idx].scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
        ax[idx].scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
        # Plot edges that came before a, b
        bool_smaller = dist.pdist(S)<=a-_tol
        edgelist = np.array([[i,j] for (i,j) in itertools.product(idx_S, idx_S) if i < j])[bool_smaller].tolist()
        bool_smaller = dist.pdist(X)<=b-_tol
        edgelist += np.array([[i,j] for (i,j) in itertools.product(range(num_points), range(num_points)) if i < j])[bool_smaller].tolist()
        for edge in edgelist:
            ax[idx].plot(X[edge][:,0], X[edge][:,1], c="black", zorder=0.5)
        # Remove axis 
        ax[idx].set_xticks([])
        ax[idx].set_yticks([])
        # Draw node labels
        for i in range(X.shape[0]):
            ax[idx].text(X[i,0]+0.05, X[i,1], f"{i}", fontsize=10)
        # end for labels 
    # end for plots
    # Plot edges from a 
    bool_smaller = dist.pdist(S)<a+_tol
    edgelist = np.array([[i,j] for (i,j) in itertools.product(idx_S, idx_S) if i < j])[bool_smaller].tolist()
    for edge in edgelist:
        ax[0].plot(X[edge][:,0], X[edge][:,1], c="black", zorder=0.5)
    # # Plot edges from b
    bool_smaller = dist.pdist(X) <b +_tol
    edgelist = np.array([[i,j] for (i,j) in itertools.product(range(num_points), range(num_points)) if i < j])[bool_smaller].tolist()
    for edge in edgelist:
        ax[2].plot(X[edge][:,0], X[edge][:,1], c="black", zorder=0.5)
    # Now, plot cycle graph of components 
    ax[3].set_xticks([])
    ax[3].set_yticks(list(range(num_points)))
    components_mat = []
    for idx in range(3):
        edgelist = ibfm_out['S_reps_0'][:pair_ab[0]-shift_ab[0]*int(idx!=0)+1]
        edgelist += ibfm_out['X_reps_0'][:pair_ab[1]-shift_ab[1]*int(idx!=2)+1]
        components = compute_components(edgelist, num_points)
        components_mat.append(components)
    
    components_mat = np.array(components_mat)
    for idx in range(3):
        u_components = np.unique(components_mat[idx]).tolist()
        points = np.array([np.ones(len(u_components))*idx, u_components]).transpose()
        ax[3].scatter(points[:,0], points[:,1], c="black", zorder=2)
        if idx==1:
            for comp in u_components:
                col_idx = components_mat[1].tolist().index(comp)
                left_comp = components_mat[0, col_idx]
                right_comp = components_mat[2, col_idx]
                ax[3].plot([0,1,2],[left_comp, comp, right_comp], c="black")
    
    
    # Adjust frames a bit more far appart
    for idx in range(4):
        xlim = ax[idx].get_xlim()
        xlength = xlim[1]-xlim[0]
        xlim = (xlim[0]-xlength*0.1, xlim[1]+xlength*0.1)
        ylim = ax[idx].get_ylim()
        ylength = ylim[1]-ylim[0]
        ylim = (ylim[0]-ylength*0.1, ylim[1]+ylength*0.1)
        ax[idx].set_xlim(xlim)
        ax[idx].set_ylim(ylim)
        if idx < 3:
            ax[idx].set_aspect("equal")
    
    # Write titles 
    ax[0].set_title(f"{a:.2f}+, {b:.2f}-")
    ax[1].set_title(f"{a:.2f}-, {b:.2f}-")
    ax[2].set_title(f"{a:.2f}-, {b:.2f}+")
    ax[3].set_title(f"G({a:.2f},{b:.2f})")


In [ ]:
S = X[S_indices]
fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(12,9))
for idx, pair_ab in enumerate([[0,3],[1,2], [0,4], [1,3]]):
    a = ibfm_out["S_barcode_0"][:,1][pair_ab[0]]
    b = ibfm_out["X_barcode_0"][:,1][pair_ab[1]]
    print(f"a:{a}, b:{b}")
    plot_geometric_matching(a, b, S_indices, X, ibfm_out, ax[idx])
plt.tight_layout()
plt.savefig(plots_dir + "matching_geometric_6pts.png")

# Additional example

Let us compute an additional example to check that the intuition about matchings and cycles holds.

In [ ]:
X = np.array([[0, 0],[1,0.3],[2,0],[3,-0.3],[4,0]])
idx_S = [0,2,4]
S = X[idx_S]
ibfm_out = ibfm.get_IBloFunMatch_subset(None, X, idx_S, output_dir, num_it=4, max_rad=-1, points=True, store_0_pm=True)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(12,3))
ax = [ax]
for idx, pair_ab in enumerate([[0,0]]):
    a = ibfm_out["S_barcode_0"][:,1][pair_ab[0]]
    b = ibfm_out["X_barcode_0"][:,1][pair_ab[1]]
    print(f"a:{a}, b:{b}")
    plot_geometric_matching(a, b, idx_S, X, ibfm_out, ax[idx], _tol=1e-3)

plt.tight_layout()
plt.savefig(plots_dir + "matching_geometric_aligned.png")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_matching(ibfm_out, ax, fig, block_function=True, dim=0)
plt.savefig(plots_dir + "block_function_0_aligned.png")

In [ ]:
ibfm_out["X_barcode_0"]

# Computation of Block Function in dimension 1

Consider the following example, with points taken from a few circles.

In [ ]:
RandGen = np.random.default_rng(2)
C0 = ibfm.sampled_circle(2.3,2.5,40, RandGen)
C1 = ibfm.sampled_circle(1,1.1,40, RandGen)-[1.1,0]
C2 = ibfm.sampled_circle(1,1.1,40, RandGen)+[1.1,0]
X = np.vstack([C0, C1, C2])
S_indices = list(range(C0.shape[0]))
S_indices += list(np.nonzero(C1[:,1]<0)[0]+C0.shape[0])
S_indices += list(np.nonzero(C2[:,1]>0)[0]+C0.shape[0]+ C1.shape[0])
S = X[S_indices]
fig, ax = plt.subplots(ncols=2, figsize=(6,3))
ax[0].scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=50, marker="o", zorder=2)
ax[1].scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x", zorder=1)
ax[0].set_axis_off()
ax[1].set_axis_off()
plt.savefig(plots_dir + "points_1.png")

In [ ]:
ibfm_out = ibfm.get_IBloFunMatch_subset(None, X, S_indices, output_dir, num_it=4, max_rad=-1, points=True, store_0_pm=True)

In [ ]:
min_length=0.3
S_barcode = ibfm_out["S_barcode_1"]
S_long = np.nonzero(S_barcode[:,1]-S_barcode[:,0] > min_length)[0].tolist()
X_barcode = ibfm_out["X_barcode_1"]
X_long = np.nonzero(X_barcode[:,1]-X_barcode[:,0] > min_length)[0].tolist()

In [ ]:
X_long

In [ ]:
S_long

In [ ]:
X_barcode_long = X_barcode[X_long]
S_barcode_long = S_barcode[S_long]

In [ ]:
ibfm_out["block_function_1"]

In [ ]:
blofun_1_long = [X_long.index(row) for col, row in enumerate(ibfm_out["block_function_1"]) if col in S_long]

In [ ]:
pm_matrix_1_long = [[X_long.index(row_idx) for row_idx in column if row_idx in X_long]  for col_idx, column in enumerate(ibfm_out["pm_matrix_1"]) if col_idx in S_long]

In [ ]:
pm_matrix_1_long 

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_from_block_function(S_barcode_long, X_barcode_long, blofun_1_long, fig, ax)
plt.savefig(plots_dir + "block_function_1.png")